# Accessing the On-street Parking Bay Sensors Data

We will access on-street parking bay sensors data provided by City of Melbourne Open Data Team. We receive all active sensor readings, however, the number of sensor readings may fluctuate as sensor connections connect/reconnect to the network. Usually, parking sensors are not operational on Public Holidays and parking sensors can show car parks as vacant when blocked by construction zones. The data is useful with caveats and updated frequently. Visit [On-street-Parking-Bay-Sensors](https://data.melbourne.vic.gov.au/Transport-Movement/On-street-Parking-Bay-Sensors/vh2v-4nfs) for more details. 

# Understanding the data

** bay_id ** - The unique ID of the parking bay where the parking sensor is located. The bay_id column is of the number datatype.

** st_marker_id ** - The street marker that is located next to the parking bay with a unique id for the bay. Often a small round, metal plaque found on the pavement next to the bay. The st_marker_id column is of the text datatype.

** status ** - The status column is of the text datatype. The status will either display:

- Present – A car is present in the parking bay at that time.

- Unoccupied – The parking bay is available at that time.

** location ** - The latitude and longitude of the parking bay. The location column is of the point datatype.

** lat ** - The latitude of the parking bay. The lat column is of the number datatype.

** lon ** - The longitude of the parking bay. The lon column is of the number datatype.


In [ ]:
# Make sure to install these packages before running (Uncomment to install):
# !pip3 install sodapy

In [ ]:
from sodapy import Socrata
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.melbourne.vic.gov.au", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.melbourne.vic.gov.au,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
result_list = client.get("dtpv-d4pf", limit=2000)

for result in result_list:
    print(result)

In [ ]:
# import statements

from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt


def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=data)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'week12'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    
    for data in result_list:
        publish_message(producer, topic, 'jsondata', data)
        sleep(1)